In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

# Fonte dos dados

In [2]:
df_selecao = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'selecoes',index_col=0)
df_jogos  = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

# Criando as funções que serão utilizadas ao longo do projeto para a resolução do problema

1. Para iniciar o projeto foi feito um escalonamento das forças das equipes. A pontuação dada pela FIFA iria diminuir a eficiência do algoritmo ao longo do execução. Assim foi feito uma matemática para que se podesse escalanor os dados entre os valores 0.15 (mín) e 1 (máx).

In [3]:
f = df_selecao["PontosRankingFIFA"]
a, b = min(f), max(f) 
fa, fb = 0.15, 1 
b1 = (fb - fa)/(b-a) 
b0 = fb - b*b1
forca = b0 + b1*f

2. Essa função tem como objetivo encontrar/prever a média de gols que cada seleção irá fazer. Tudo isso se baseando no bloco anterior, ou seja, segundo os dados, escalonados, da fifa. Sendo assim, a expectativa de gol para cada partida foi de 2.63, segundo o site, essa informção está contando até o o jogo do dia 14/12, jogo das quartas de finais entre Marrocos e França, sendo a França a vencedora.

    Refêrencia: https://www.goal.com/br/not%C3%ADcias/quantos-gols-sairam-na-copa-do-mundo-2022-e-qual-a-media-por-jogo/bltf039878c7bfeddcc

In [4]:
def media_equipes(selecao1, selecao2):
    forca_casa = forca[f'{selecao1}']
    forca_fora = forca[f'{selecao2}']
    expectativa_de_gols = 2.63
    lambda_casa = expectativa_de_gols*forca_casa/(forca_casa + forca_fora)
    lambda_fora = expectativa_de_gols*forca_fora/(forca_fora + forca_fora)
    return [lambda_casa, lambda_fora]

3. Utilizando do conhecimento da **Distribuição de Poisson** é possivel encontrar as probabilidades de gols de cada time. A melhor forma para visulizar essa informação é através de uma matriz, já que iremos analisar a probabilidade de gols das duas equipes simultaneamente. Sendo que *o melhor resultado será aquele o qual a soma das probabilidade das duas equipes é maior.*

    "A distribuição de Poisson é uma distribuição de probabilidade de variável aleatória discreta que expressa a probabilidade de uma série de eventos ocorrer num certo período de tempo se estes eventos ocorrem independentemente de quando ocorreu o último evento."

    Refêrencia: https://pt.wikipedia.org/wiki/Distribui%C3%A7%C3%A3o_de_Poisson

In [5]:
def distribuicao(media):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i,media))
    probs.append(1-sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])

4. Nesse bloco foi feito para encontrar a quantidade de gols que uma seleção irá fazer na outra, segundo a média equipe, usando a distribuição de poisson, ou seja, a quantidade de gols que a equipe é esperada fazer. 

In [6]:
def qtd_gols(casa,fora):
    lambda_casa, lambda_fora = media_equipes(selecao1=casa,selecao2=fora)
    gols_casa = np.random.poisson(lam=lambda_casa,size=1)
    gols_fora = np.random.poisson(lam=lambda_fora,size=1)
    saldo_time_casa = gols_casa - gols_fora
    saldo_time_fora = -saldo_time_casa
    placar = f'{saldo_time_casa}x{saldo_time_fora}'
    return [gols_casa, gols_fora, saldo_time_casa, saldo_time_fora, placar]

5. Na próxima função foi criada com a finalidade de encontrar as "probabilidades" das, respectivas, seleções. Com essa probabilidade encontraremos o resultado do jogo e a matriz que irá informar qual será a possivel combinação, de gols, da partida.

In [7]:
def probabilidade(selecao1,selecao2):
    lambda_casa, lambda_fora = media_equipes(selecao1=selecao1, selecao2=selecao2)
    dist1, dist2 = distribuicao(media = lambda_casa), distribuicao(media= lambda_fora)
    matriz = np.outer(dist1, dist2)
    vitoria = np.tril(matriz).sum()-np.trace(matriz)    #Soma a triangulo inferior
    derrota = np.triu(matriz).sum()-np.trace(matriz)    #Soma a triangulo superior
    empate = 1 - (vitoria + derrota)
    probs = np.around([vitoria, empate , derrota], 3)
    
    probsp = [f'{100*i:.1f}%' for i in probs]

    nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns]) 

    output = {'seleção1': selecao1, 'seleção2': selecao2, 
             'f1': forca[selecao1], 'f2': forca[selecao2], 
             'media1': lambda_casa, 'media2': lambda_fora, 
             'probabilidades': probsp, 'matriz': matriz}
    
    return output

6. Função tem como ojetivo classificar, segundo a quantidade de gol, se houve uma vítoria, empate ou derrota

In [8]:
def Resultado(gols1, gols2):
    if gols1 > gols2:
        res = 'V'
    if gols1 < gols2:
        res = 'D' 
    if gols1 == gols2:
        res = 'E'       
    return res

7. Usando a informação da função anterior. A seguinte função tem como produto contar a quantidade de pontos que cada equipe fez. Seguindo o padrão básico do futebol. Vitória = 3 pontos; Empate = 1; Derrota = 0.

In [9]:
def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
        pontos1, pontos2 = 3, 0
    if rst == 'E':
        pontos1, pontos2 = 1, 1
    if rst == 'D':
        pontos1, pontos2 = 0, 3
    return pontos1, pontos2, rst

8. No seguinte bloco de código foi construida uma função para que se possa identificar o resultado da partida. Como pode ser visto na última linha do bloco, temos um retorno dos gols feitos pela equipe 1 e 2, saldo da equipe 1 e 2, o resultado (pontuação) e o placar do time 1 e 2.

In [10]:
def Jogo(selecao1, selecao2):
    l1, l2 = media_equipes(selecao1, selecao2)
    gols1 = int(np.random.poisson(lam = l1, size = 1))
    gols2 = int(np.random.poisson(lam = l2, size = 1))
    saldo1 = gols1 - gols2
    saldo2 = -saldo1
    pontos1, pontos2, result = Pontos(gols1, gols2)
    placar = '{}x{}'.format(gols1, gols2)
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, result, placar]

10. No seguinte bloco tem a finalidade de organizar as probabilidades de cada time em uma matriz. No caso, a probabilidade de vitória, empate ou derrota de cada time e, claro, usando as funções criadas anteriormente.

In [11]:
df_jogos['Vitoria'] = None
df_jogos['Empate'] = None
df_jogos['Derrota'] = None

for i in range(len(df_jogos)):
    s1 = df_jogos['seleção1'][i]
    s2 = df_jogos['seleção2'][i]
    v,e,d = probabilidade(selecao1=s1, selecao2=s2)['probabilidades']
    df_jogos.at[i,'Vitoria'] = v
    df_jogos.at[i,'Empate'] = e
    df_jogos.at[i,'Derrota'] = d

# Fase de Grupos

11. Trazendo para a prática todas as funções, iremos, no próximo bloco de código, criar o "esqueleto" de como funciona a fase de grupos da copa.

    Assim sendo, na fase de grupos , "as equipes competem em oito grupos de quatro equipes cada. Oito equipes são as cabeças de chave (incluindo os anfitriões  e as outras equipes que são selecionadas usando o ranking da FIFA como parâmetro, posicionado-os em grupos separados. As outras equipes são adicionadas geralmente com base em critérios geográficos , e as equipes em cada pote são sorteadas aleatoriamente para os oito grupos."

    "Cada grupo tem um torneio round-robin , garantindo que cada equipe irá jogar pelo menos três partidas . As duas melhores equipes de cada grupo avançam para a fase eliminatória . Os pontos são usados ​​para classificar as equipes dentro de um grupo. Desde 1994, três pontos foram concedidos para uma vitória , um por empate e nenhum para uma derrota (antes disso, os vencedores recebiam dois pontos em vez de três). Se duas ou mais equipes acabam com o mesmo número de pontos, são usados ​​critérios de desempate, o mais importante é o saldo de gols."

    Refência: https://www.wikiartigos.com.br/como-funciona-a-copa-do-mundo-da-fifa/#:~:text=Na%20fase%20de%20grupos%2C%20as%20equipes%20competem%20em,da%20FIFA%20como%20par%C3%A2metro%2C%20posicionado-os%20em%20grupos%20separados.

In [12]:
def JogosGrupo(dados, grupo):
    
    times = list(dados.loc[dados['Grupo']==grupo].index)

    time1, time2, time3, time4 = times

    pt1, pt2, pt3, pt4 = 0, 0, 0, 0
    gp1, gp2, gp3, gp4 = 0, 0, 0, 0
    sg1, sg2, sg3, sg4 = 0, 0, 0, 0

    jogo1 = Jogo(time1, time2)
    jogo2 = Jogo(time3, time4)

    jogo3 = Jogo(time1, time3)
    jogo4 = Jogo(time2, time4)

    jogo5 = Jogo(time1, time4)
    jogo6 = Jogo(time2, time3)

    gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[0], gp2 + jogo1[1], sg1 + jogo1[2], sg2 + jogo1[3], pt1 + jogo1[4], pt2 + jogo1[5]
    gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[0], gp4 + jogo2[1], sg3 + jogo2[2], sg4 + jogo2[3], pt3 + jogo2[4], pt4 + jogo2[5]
    gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[0], gp3 + jogo3[1], sg1 + jogo3[2], sg3 + jogo3[3], pt1 + jogo3[4], pt3 + jogo3[5]
    gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[0], gp4 + jogo4[1], sg2 + jogo4[2], sg4 + jogo4[3], pt2 + jogo4[4], pt4 + jogo4[5]
    gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[0], gp4 + jogo5[1], sg1 + jogo5[2], sg4 + jogo5[3], pt1 + jogo5[4], pt4 + jogo5[5]
    gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[0], gp3 + jogo6[1], sg2 + jogo6[2], sg3 + jogo6[3], pt2 + jogo6[4], pt3 + jogo6[5]

    partidas = [ time1 + ' x ' + time2, 
                 time3 + ' x ' + time4,
                 time1 + ' x ' + time3, 
                 time2 + ' x ' + time4,
                 time1 + ' x ' + time4,
                 time2 + ' x ' + time3 ]

    resultados = [ jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6] ]
    placares = [ jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1] ] 
    cols = ['Pontos', 'Saldo de Gols', 'Gols Pró']
    tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp4]], index = cols, columns = times).transpose()
    
    tab = tab.sort_values(['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending = False)
    tab['Posição'] = [1, 2, 3, 4]

    jogos = pd.DataFrame([partidas, placares, resultados]).transpose()
    jogos.columns = ['Partida', 'Placar', 'Resultado']

    return [tab, jogos]

# Fase eliminatória

12. Encontrado as 16 seleções que tiveram a melhor performace nessa fase de grupos, iremos utilizar de uma função para identificar como ficará a fase, mais conhecida no Brasil, como "Mata-Mata".

    "A fase eliminatória é um torneio de eliminação simples em que as equipes jogam entre si em jogos únicos, com tempo extra e pênaltis usados ​​para decidir o vencedor, se necessário. Ela começa com a “rodada dos 16”, em que o vencedor de cada grupo joga contra o segundo colocado do outro grupo (primeiro do A contra o segundo do B etc), seguido pelas quartas de final , as semi-finais, o jogo do terceiro lugar ( contestado pelos perdedores semifinalistas ) , e a grande final."

    Uma observação a ser feita no algoritmo a seguir é que em caso de empate, foi usado o "random.sample" um algoritmo que irá decidir de forma aleatória o vencedor da partida, já que, penaltis tem uma pequena relação com a sorte...

    Refência: https://www.wikiartigos.com.br/como-funciona-a-copa-do-mundo-da-fifa/#:~:text=Na%20fase%20de%20grupos%2C%20as%20equipes%20competem%20em,da%20FIFA%20como%20par%C3%A2metro%2C%20posicionado-os%20em%20grupos%20separados.
    

In [13]:
def JogoMM(selecao1, selecao2):
    r = Jogo(selecao1=selecao1,selecao2=selecao2)
    if r[6] == 'V':
        return selecao1
    elif r[6] == 'D':
        return selecao2
    else:
        return random.sample([selecao1, selecao2], 1)[0]

13. Definida a solução para representar os jogos "Mata-Mata" iremos utiliza-la para encontrar a simular todas as probabilides das seleções que estão na copa.

    Nesse algoritmo iremos identificar qual será as probabilides, de cada seleção, de forma individual, em ser o 1°, o 2°, 3° e 4° do seu grupo. Em seguida, apartir da fase eliminatória, qual será a chances do time ir passar para a proxima fase. Por fim, qual é a real chance dessa selação ser a campeã.

In [14]:
def SimulaCopa(dados):

    # Data Frame que será usado para guardar as informações
    cols = ['1st', '2nd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
    n = dados.shape[0]
    m = len(cols)
    aux = np.array(np.zeros(n*m).reshape(n, m))
    info = pd.DataFrame(aux, columns = cols, index = dados.index) 
    info = info.astype(int)

    # As seleções que passaram para a proxima fase
    top16 = []
    for i in ['A','B','C','D','E','F','G','H']:
        resultados = JogosGrupo(dados=dados,grupo=i)[0]
        top16+=resultados.index[:2].to_list()
        anomes = resultados.index.to_list()
        info.at[anomes[0], '1st'] = 1
        info.at[anomes[1], '2nd'] = 1
        info.at[anomes[2], '3rd'] = 1
        info.at[anomes[3], '4th'] = 1
    
    # Oitavas de finais
    qf1 = JogoMM(top16[0], top16[3])   #A1 x B2
    qf2 = JogoMM(top16[2], top16[1])   #B1 x A2
    qf3 = JogoMM(top16[4], top16[7])   #C1 x D2
    qf4 = JogoMM(top16[6], top16[5])   #D1 x C2
    qf5 = JogoMM(top16[8], top16[11])  #E1 x F2
    qf6 = JogoMM(top16[10], top16[9])  #F1 x E2
    qf7 = JogoMM(top16[12], top16[15]) #G1 x H2
    qf8 = JogoMM(top16[14], top16[13]) #H1 x G2

    # Quartas de finais
    top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]
    sf1 = JogoMM(qf1, qf3)
    sf2 = JogoMM(qf2, qf4) 
    sf3 = JogoMM(qf5, qf7) 
    sf4 = JogoMM(qf6, qf8) 

    # Semi-finais
    top4 = [sf1, sf2, sf3, sf4]
    f1 = JogoMM(sf1, sf3) 
    f2 = JogoMM(sf2, sf4) 
    
    # Final
    top2 = [f1, f2]
    top1 = JogoMM(f1,f2)

    # Atualizando o data frame
    for i in range(len(top16)):
        info.at[top16[i], 'Oitavas'] = 1

    for i in range(len(top8)):
        info.at[top8[i], 'Quartas'] = 1

    for i in range(len(top4)):
        info.at[top4[i], 'Semis'] = 1

    for i in range(len(top2)):
        info.at[top2[i], 'Final'] = 1    

    info.at[top1, 'Campeão'] = 1

    return info

14. Por fim, foi criado esse algoritmo simples para simular a copa n vezes. Temos que fazer isso, simular n vezes, pois, somente uma ou 10 vezes calculada a copa existirá uma certa instabilidade no resultado final. Assim, pouco provavel de ser o real resultado da copa.   

In [15]:
def Toda_a_Copa(dados, qtd):
    info = SimulaCopa(dados=dados)
    for i in (range(qtd-1)):
        info += SimulaCopa(df_selecao)
    return info.sort_values(by='Campeão', ascending=False)/qtd

# Conclusão

É recomendado que a simulação seja executada, no mínimo 1.000.000 de vezes. Mas, caso não tenha um computador bom, assim como o meu, rode 1000 vezes, lembrando que esse resultado não será muito consistente.

In [16]:
resultado_da_copa = Toda_a_Copa(dados=df_selecao, qtd=1000)

Foi utilizado para definir quem será o vencedor da copa de 2022 aquela seleção que tiver o maior probabalidade para ser campeão ao fazer a simualação. O Data Frame que será gerado já está ordenado de acordo com o maior resultado para o menor resultado.

Se o Brasil perdeu, não desanime é só esperar mais 4 anos... Fica "tite" não!

In [18]:
resultado_da_copa.head()

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Brasil,0.408,0.264,0.210,0.118,0.672,0.404,0.249,0.142,0.080
Argentina,0.393,0.282,0.189,0.136,0.675,0.403,0.236,0.137,0.079
Bélgica,0.381,0.275,0.197,0.147,0.656,0.359,0.194,0.124,0.069
Inglaterra,0.344,0.251,0.222,0.183,0.595,0.349,0.187,0.106,0.063
França,0.370,0.262,0.225,0.143,0.632,0.340,0.179,0.091,0.058


Aqui teremos o resultado final geral da simualção que foi feita por você. Verifique a pasta onde foi executada esse arquivo .ipynb.

In [17]:
resultado_da_copa.to_csv('resultado.csv')